<a href="https://colab.research.google.com/github/nilavya2000/transformer/blob/main/transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import tensorflow as tf
import tensorflow_datasets as tfds 
import tensorflow_text as text


import numpy as np
import matplotlib.pyplot as plt

import collections
import os
import logging
import pathlib
import string
import re
import sys
import time 

In [4]:
logging.getLogger('tensorlfow').setLevel(logging.ERROR)

In [6]:
#loading the datasets
examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True, as_supervised=True )
train_examples, val_examples = examples['train'], examples['validation']


In [10]:
for pt_examples, en_example in train_examples.batch(3).take(1):
  for pt in pt_examples.numpy():
    print(pt.decode('utf-8'))
  print("\n")
  for en in en_example.numpy():
    print(en.decode('utf-8'))


e quando melhoramos a procura , tiramos a única vantagem da impressão , que é a serendipidade .
mas e se estes fatores fossem ativos ?
mas eles não tinham a curiosidade de me testar .


and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


*Text tokenization and detokinization*

In [14]:
model_name = "ted_hrlr_translate_pt_en_converter"
tf.keras.utils.get_file(
    f"{model_name}.zip",
    f'https://storage.googleapis.com/download.tensorflow.org/models/{model_name}.zip' ,
    cache_dir='.', cache_subdir='', extract=True
)

188416/184801 [==============================] - 0s 0us/step


'./ted_hrlr_translate_pt_en_converter.zip'

In [16]:
tokenizer=tf.saved_model.load(model_name)

In [18]:
[item for item in dir(tokenizer.en) if not item.startswith('_')]

['detokenize',
 'get_reserved_tokens',
 'get_vocab_path',
 'get_vocab_size',
 'lookup',
 'tokenize',
 'tokenizer',
 'vocab']

In [19]:
for en in en_example.numpy():
  print(en.decode('utf-8'))

and when you improve searchability , you actually take away the one advantage of print , which is serendipity .
but what if it were active ?
but they did n't test for curiosity .


In [22]:
encode = tokenizer.en.tokenize(en_example)
for row in encode.to_list():
  print(row)

[2, 72, 117, 79, 1259, 1491, 2362, 13, 79, 150, 184, 311, 71, 103, 2308, 74, 2679, 13, 148, 80, 55, 4840, 1434, 2423, 540, 15, 3]
[2, 87, 90, 107, 76, 129, 1852, 30, 3]
[2, 87, 83, 149, 50, 9, 56, 664, 85, 2512, 15, 3]


In [27]:
round_trip=tokenizer.en.detokenize(encode)
for line in round_trip.numpy():
  print("* ",line)
  print()
  print("->> ",line.decode('utf-8'))
  print()

*  b'and when you improve searchability , you actually take away the one advantage of print , which is serendipity .'

->>  and when you improve searchability , you actually take away the one advantage of print , which is serendipity .

*  b'but what if it were active ?'

->>  but what if it were active ?

*  b"but they did n ' t test for curiosity ."

->>  but they did n ' t test for curiosity .



In [28]:
#lower_level lookup converts from token-IDS to token text

tokens = tokenizer.en.lookup(encode)
print("tokens \n ", tokens)

tokens 
  <tf.RaggedTensor [[b'[START]', b'and', b'when', b'you', b'improve', b'search', b'##ability', b',', b'you', b'actually', b'take', b'away', b'the', b'one', b'advantage', b'of', b'print', b',', b'which', b'is', b's', b'##ere', b'##nd', b'##ip', b'##ity', b'.', b'[END]'], [b'[START]', b'but', b'what', b'if', b'it', b'were', b'active', b'?', b'[END]'], [b'[START]', b'but', b'they', b'did', b'n', b"'", b't', b'test', b'for', b'curiosity', b'.', b'[END]']]>


*Set input pipeline*

In [29]:
def tokenize_pairs(pt, en):
  pt=tokenizer.pt.tokenize(pt)
  pt=pt.to_tensor()
  
  en=tokenizer.en.tokenize(en)
  en=en.to_tensor()

  return pt, en

In [30]:
BUFFER_SIZE = 2000
BATCH_SIZE = 64


In [31]:
def make_batch(ds):
  return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE)
  )

train_batchs = make_batch(train_examples)
val_batches = make_batch(val_examples)

In [32]:
train_batchs

<PrefetchDataset shapes: ((None, None), (None, None)), types: (tf.int64, tf.int64)>